In [1]:
import orca

## Register data tables

This exercise starts from a point where data tables are already registered with orca. 

Here I'm using some San Diego data from Paul Sohn's [urbansim_parcels](https://github.com/urbansim/urbansim_parcels) project. I moved over the minimum code needed to begin estimating real estate price models.

Importing 'datasources.py' and 'models.py' registers a variety of functions as orca objects.

In [2]:
import os; os.chdir('../legacy')
import datasources
import models

In [3]:
orca.list_tables()

['parcels']

In [4]:
orca.get_table('parcels').to_frame().describe()

,development_type_id,land_value,parcel_acres,county_id,mgra_id,zoning_id,luz_id,msa_id,proportion_undevelopable,tax_exempt,...,distance_to_freeway,distance_to_onramp,distance_to_park,distance_to_school,distance_to_transit,x,y,taz_id,node_id,acres
count,28697.000000,2.869700e+04,28697.000000,28697.0,28697.000000,28697.000000,28697.000000,28697.000000,28697.0,28697.000000,...,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000,28697.000000
mean,19.731331,1.587254e+05,0.288399,1.0,984.686413,384.363348,92.602572,0.000801,0.0,0.149493,...,2076.871550,2091.113508,1514.817128,1381.253821,8384.150568,-117.123172,32.744595,3528.548455,44453.184584,0.288399
std,4.000490,3.563180e+05,6.351003,0.0,780.646635,49.787739,5.303660,0.028300,0.0,0.356580,...,1327.156982,1275.229073,857.112030,661.389162,2722.241600,0.015923,0.013887,214.671611,4229.966661,6.351003
min,2.000000,0.000000e+00,0.000231,1.0,163.000000,242.000000,58.000000,0.000000,0.0,0.000000,...,1.713400,2.183857,0.000000,1.945715,70.392898,-117.157931,32.715194,3129.000000,36353.000000,0.000231
25%,19.000000,2.981300e+04,0.100923,1.0,431.000000,356.000000,88.000000,0.000000,0.0,0.000000,...,983.660053,1089.847212,858.845876,870.745945,6297.968560,-117.133901,32.734404,3337.000000,41422.000000,0.100923
50%,19.000000,1.051070e+05,0.141347,1.0,659.000000,403.000000,92.000000,0.000000,0.0,0.000000,...,1839.571534,1841.138876,1443.148884,1313.105024,8426.799925,-117.123128,32.745939,3518.000000,43946.000000,0.141347
75%,21.000000,2.079860e+05,0.176792,1.0,1905.000000,423.000000,93.000000,0.000000,0.0,0.000000,...,3035.392381,2932.979913,2068.329358,1808.525493,10563.229751,-117.110015,32.756466,3698.000000,47886.000000,0.176792
max,31.000000,3.375000e+07,1058.794862,1.0,6194.000000,460.000000,110.000000,1.000000,0.0,1.000000,...,6014.347930,5917.269942,4638.489174,3400.000000,13671.494235,-117.094652,32.768020,4044.000000,52612.000000,1058.794862
